# Introduction

In this project for MegaLines telecom company, we have been tasked to help identify which of the two plans offered to the customers is more profitable. This will in turn help to inform the marketing team on how to adjust the budget for advertising.  

Using a small sample of 500 customers and their behavior in regards to messaging, calls and Internet usage from the year 2018. we will conduct a preliminary analysis to investigate the revenue distribution of customers in each plan. 

## Changes for Version-1.0 

- Organize this notebook, readable and consistent. 

- changes will be note in **Comment:**

- Improve the EDA and charting, Update methods. and simplify the process and findings. 

## Libraries 

In [1]:
import pandas as pd
import scipy as py
from scipy import stats as st 
import numpy as np
import matplotlib.pyplot as plt
import math
import statistics as stats

## Load Datasets

**DataSets**
keys for all data sets "user_id"

"id" represent unique instances for each user. ex: '1000'+'_93' 'user_id'+ '_instanceid'

`calls_data` - multiple values for calls for each 'user_id'.

- "duration" is the time of the call. each call has a date in "call_date"
- 137735 rows total for the 500 unique "user_ids"
<br>

`messages_data` - messages for each 'user_id'
- the count of messages are are in the number of unique "id" per 'user_id'
<br>

`internet_data` - megabites uses per session in each 'id'
<br>

`plans_data` - Metrics for both plans. 
- limits for each plan
- price for overages
<br>

`user_data`
 - 'first_name' & 'last_name' 
 - 'age'
 - 'city', contains city and state
 - 'reg_date', data of registration
 - 'tariff', plan for this 'user_id'
 - 'churn_date' - "nan" values for current users

In [2]:
# Load the data files into different DataFrames
calls_data = pd.read_csv(
    '/Users/ericross/Desktop/Professional_Projects/Mega-Lines-project/Megaline datasets/megaline_calls.csv',
    parse_dates=['call_date']
)

In [3]:
messages_data = pd.read_csv(
    '/Users/ericross/Desktop/Professional_Projects/Mega-Lines-project/Megaline datasets/megaline_messages.csv',
    parse_dates=['message_date']
)

In [4]:
internet_data = pd.read_csv(
    '/Users/ericross/Desktop/Professional_Projects/Mega-Lines-project/Megaline datasets/megaline_internet.csv',
    parse_dates=['session_date']
)

In [5]:
plans_data = pd.read_csv(
    '/Users/ericross/Desktop/Professional_Projects/Mega-Lines-project/Megaline datasets/megaline_plans.csv'
)

In [6]:
users_data = pd.read_csv(
    '/Users/ericross/Desktop/Professional_Projects/Mega-Lines-project/Megaline datasets/megaline_users1.csv',
    parse_dates=['reg_date']
)

# Preprocessing

## Plans_data

In [7]:
# Print a sample of data for plans
plans_data


,messages_included,mb_per_month_included,minutes_included,usd_monthly_pay,usd_per_gb,usd_per_message,usd_per_minute,plan_name
0,50,15360,500,20,10,0.03,0.03,surf
1,1000,30720,3000,70,7,0.01,0.01,ultimate


## Users_data

In [8]:
# Print the general/summary information about the users' DataFrame
users_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     500 non-null    int64         
 1   first_name  500 non-null    object        
 2   last_name   500 non-null    object        
 3   age         500 non-null    int64         
 4   city        500 non-null    object        
 5   reg_date    500 non-null    datetime64[ns]
 6   tariff      500 non-null    object        
 7   churn_date  34 non-null     object        
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 31.4+ KB


In [9]:
users_data.rename(columns={'city': 'location'}, inplace=True)


In [10]:
print(users_data['user_id'].duplicated().sum())

users_data['user_id'].unique()

0


array([1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010,
       1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021,
       1022, 1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032,
       1033, 1034, 1035, 1036, 1037, 1038, 1039, 1040, 1041, 1042, 1043,
       1044, 1045, 1046, 1047, 1048, 1049, 1050, 1051, 1052, 1053, 1054,
       1055, 1056, 1057, 1058, 1059, 1060, 1061, 1062, 1063, 1064, 1065,
       1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1076,
       1077, 1078, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087,
       1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1096, 1097, 1098,
       1099, 1100, 1101, 1102, 1103, 1104, 1105, 1106, 1107, 1108, 1109,
       1110, 1111, 1112, 1113, 1114, 1115, 1116, 1117, 1118, 1119, 1120,
       1121, 1122, 1123, 1124, 1125, 1126, 1127, 1128, 1129, 1130, 1131,
       1132, 1133, 1134, 1135, 1136, 1137, 1138, 1139, 1140, 1141, 1142,
       1143, 1144, 1145, 1146, 1147, 1148, 1149, 11

In [11]:
users_data['reg_date'].min(),users_data['reg_date'].max()


(Timestamp('2018-01-01 00:00:00'), Timestamp('2018-12-31 00:00:00'))

In [12]:
import re

users_data['state'] = users_data['location'].str.extract(r',\s*([A-Za-z]+(?:\s*-?[A-Za-z]+)*)')
users_data['state'] = users_data['state'].str.replace(r'\s+MSA$', '')

users_data['state'].value_counts()

/var/folders/2v/vvfftz_j38j68jttxc75xyf00000gn/T/ipykernel_32813/2039806083.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  users_data['state'] = users_data['state'].str.replace(r'\s+MSA$', '')


NY-NJ-PA       80
CA             78
TX             39
FL             25
IL-IN-WI       19
PA-NJ-DE-MD    17
MI             16
GA             14
WA             13
TN             12
MA-NH          12
DC-VA-MD-WV    11
MN-WI          11
AZ             11
OH              9
LA              9
CO              9
OR-WA           8
NC-SC           8
OH-KY-IN        8
SC              7
NV              7
CT              6
NY              6
IN              6
OK              6
PA              5
KY-IN           5
HI              5
AL              4
VA-NC           4
MD              4
VA              4
RI-MA           3
UT              3
WI              3
MO-IL           3
NM              2
MO-KS           2
NE-IA           2
TN-MS-AR        2
NC              2
Name: state, dtype: int64

In [13]:
users_data['month'] = users_data['reg_date'].dt.month
users_data.head()

,user_id,first_name,last_name,age,location,reg_date,tariff,churn_date,state,month
0,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaN,GA,12
1,1001,Mickey,Wilkerson,28,"Seattle-Tacoma-Bellevue, WA MSA",2018-08-13,surf,NaN,WA,8
2,1002,Carlee,Hoffman,36,"Las Vegas-Henderson-Paradise, NV MSA",2018-10-21,surf,NaN,NV,10
3,1003,Reynaldo,Jenkins,52,"Tulsa, OK MSA",2018-01-28,surf,NaN,OK,1
4,1004,Leonila,Thompson,40,"Seattle-Tacoma-Bellevue, WA MSA",2018-05-23,surf,NaN,WA,5


In [14]:
users_data['tariff_id'] = np.where(users_data['tariff'] == 'ultimate', 1, 0)

In [15]:
users_data['tariff_id'].value_counts()

0    339
1    161
Name: tariff_id, dtype: int64

 **Comment:** plot the share of plans in the data, with a Pie chart. 

### Fix Churn Date columns

In [16]:
users_data.churn_date.fillna(users_data['reg_date'].max(),inplace=True)

users_data['churn'] =  np.where(users_data['churn_date'] == users_data['reg_date'].max(), 0, 1)

users_data['churn'].value_counts()

0    466
1     34
Name: churn, dtype: int64

### Remove Unneeded columns form Users_Data

In [17]:
del users_data['tariff']
del users_data['reg_date']
del users_data['churn_date']
del users_data['location']

In [18]:
users_data.head()

,user_id,first_name,last_name,age,state,month,tariff_id,churn
0,1000,Anamaria,Bauer,45,GA,12,1,0
1,1001,Mickey,Wilkerson,28,WA,8,0,0
2,1002,Carlee,Hoffman,36,NV,10,0,0
3,1003,Reynaldo,Jenkins,52,OK,1,0,0
4,1004,Leonila,Thompson,40,WA,5,0,0


## Calls_data 

In [19]:
# Print the general/summary information about the calls' DataFrame
calls_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137735 entries, 0 to 137734
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   id         137735 non-null  object        
 1   user_id    137735 non-null  int64         
 2   call_date  137735 non-null  datetime64[ns]
 3   duration   137735 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 4.2+ MB


**Comment:** useful info needed from calls_data 

- total and average call duration for each user. 

- add a call_count for each user. 



In [20]:
calls_data['user_id'].unique()

array([1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010,
       1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021,
       1022, 1023, 1024, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033,
       1034, 1035, 1036, 1037, 1038, 1039, 1040, 1041, 1042, 1043, 1044,
       1045, 1046, 1047, 1048, 1049, 1050, 1051, 1052, 1053, 1054, 1055,
       1056, 1057, 1058, 1059, 1060, 1061, 1062, 1063, 1064, 1065, 1066,
       1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1076, 1077,
       1078, 1079, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1090,
       1091, 1092, 1093, 1094, 1095, 1096, 1097, 1098, 1099, 1100, 1101,
       1102, 1103, 1104, 1105, 1106, 1107, 1109, 1110, 1111, 1112, 1113,
       1114, 1115, 1116, 1117, 1118, 1119, 1120, 1122, 1123, 1124, 1125,
       1126, 1127, 1128, 1130, 1132, 1133, 1134, 1135, 1136, 1138, 1139,
       1140, 1141, 1142, 1144, 1145, 1146, 1147, 1148, 1149, 1150, 1151,
       1152, 1153, 1154, 1155, 1156, 1157, 1158, 11

### Enrich data

In [21]:
calls_data['month'] = calls_data['call_date'].dt.month

In [22]:
calls_data['day'] = calls_data['call_date'].dt.day

In [23]:
calls_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137735 entries, 0 to 137734
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   id         137735 non-null  object        
 1   user_id    137735 non-null  int64         
 2   call_date  137735 non-null  datetime64[ns]
 3   duration   137735 non-null  float64       
 4   month      137735 non-null  int64         
 5   day        137735 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 6.3+ MB


In [24]:
calls_data.sample(5)

,id,user_id,call_date,duration,month,day
62079,1223_248,1223,2018-12-23,7.35,12,23
21635,1079_348,1079,2018-10-12,9.39,10,12
10797,1048_219,1048,2018-09-19,3.50,9,19
86563,1320_232,1320,2018-08-22,8.03,8,22
130997,1470_446,1470,2018-10-19,3.45,10,19


## Messages_data

In [25]:
# Print the general/summary information about the messages' DataFrame

messages_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76051 entries, 0 to 76050
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            76051 non-null  object        
 1   user_id       76051 non-null  int64         
 2   message_date  76051 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 1.7+ MB


In [26]:
# Print a sample of data for messages

messages_data.sample(5)

,id,user_id,message_date
10363,1076_831,1076,2018-09-10
48788,1326_325,1326,2018-12-03
48064,1324_794,1324,2018-05-19
72238,1468_75,1468,2018-12-03
50265,1328_1223,1328,2018-08-14


### Enrich data

In [27]:
messages_data['month'] = messages_data['message_date'].dt.month

In [28]:
messages_data['message_day'] = messages_data['message_date'].dt.day

In [29]:
messages_data.sample(5)

,id,user_id,message_date,month,message_day
17675,1114_775,1114,2018-08-06,8,6
71137,1464_248,1464,2018-12-31,12,31
1524,1015_265,1015,2018-12-16,12,16
70491,1458_197,1458,2018-12-01,12,1
29761,1183_156,1183,2018-12-18,12,18


## Internet_data

In [30]:
# Print the general/summary information about the internet DataFrame

internet_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104825 entries, 0 to 104824
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   id            104825 non-null  object        
 1   user_id       104825 non-null  int64         
 2   session_date  104825 non-null  datetime64[ns]
 3   mb_used       104825 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 3.2+ MB


In [31]:
# Print a sample of data for the internet traffic

internet_data.head()

,id,user_id,session_date,mb_used
0,1000_13,1000,2018-12-29,89.86
1,1000_204,1000,2018-12-31,0.00
2,1000_379,1000,2018-12-28,660.40
3,1000_413,1000,2018-12-26,270.99
4,1000_442,1000,2018-12-27,880.22


### Enrich data

In [32]:
internet_data['month'] = internet_data['session_date'].dt.month

In [33]:
internet_data['day'] = internet_data['session_date'].dt.day

In [34]:
internet_data.head()

,id,user_id,session_date,mb_used,month,day
0,1000_13,1000,2018-12-29,89.86,12,29
1,1000_204,1000,2018-12-31,0.00,12,31
2,1000_379,1000,2018-12-28,660.40,12,28
3,1000_413,1000,2018-12-26,270.99,12,26
4,1000_442,1000,2018-12-27,880.22,12,27


# EDA

## Study plan conditions

In [79]:
# Print out the plan conditions and make sure they are clear for you
plans_data['gb_per_month_included'] = plans_data['mb_per_month_included']/1000

plans_data

,messages_included,mb_per_month_included,minutes_included,usd_monthly_pay,usd_per_gb,usd_per_message,usd_per_minute,plan_name,gb_per_month_included
0,50,15360,500,20,10,0.03,0.03,surf,15.36
1,1000,30720,3000,70,7,0.01,0.01,ultimate,30.72


### calls_data analysis

##### User_id per month 

In [43]:
#grouping users and month by duration
calls_grouped = calls_data.groupby(['user_id', 'month']).agg(monthly_duration=('duration', 'sum'))
calls_grouped.head(10)

monthly_duration
user_id month                  
1000    12               116.83
1001    8                171.14
        9                297.69
        10               374.11
        11               404.59
        12               392.93
1002    10                54.13
        11               359.76
        12               363.24
1003    12              1041.00

In [44]:
total_calls = calls_data.groupby(['user_id', 'month']).agg(total_calls=('id', 'count'))
total_calls.head(10)

total_calls
user_id month             
1000    12              16
1001    8               27
        9               49
        10              65
        11              64
        12              56
1002    10              11
        11              55
        12              47
1003    12             149

In [39]:
messages_grouped = messages_data.groupby(['user_id', 'month']).agg(monthly_messages=('id', 'count'))
messages_grouped.head(10)

monthly_messages
user_id month                  
1000    12                   11
1001    8                    30
        9                    44
        10                   53
        11                   36
        12                   44
1002    10                   15
        11                   32
        12                   41
1003    12                   50

In [80]:
internet_grouped = internet_data.groupby(['user_id', 'month']).agg(monthly_mb=('mb_used', 'sum'))
internet_grouped['monthly_gb'] = internet_grouped['monthly_mb']/1000
internet_grouped.head()

monthly_mb  monthly_gb
user_id month                        
1000    12        1901.47     1.90147
1001    8         6919.15     6.91915
        9        13314.82    13.31482
        10       22330.49    22.33049
        11       18504.30    18.50430

In [46]:
users_plan = users_data.groupby(['user_id','month']).agg(state=('state', 'sum'),plan=('tariff_id', 'sum') )
users_plan.head()

,,state,plan
user_id,month,,
1000,12,GA,1
1001,8,WA,0
1002,10,NV,0
1003,1,OK,0
1004,5,WA,0


##### Stats for customers by month. 

In [83]:
monthly_stats = pd.concat([calls_grouped, messages_grouped, internet_grouped, total_calls, users_plan], axis=1, keys=['calls', 'messages', 'internet', 'total_calls', 'users_plan_1'])
monthly_stats.columns = monthly_stats.columns.droplevel(0)
monthly_stats.isna().sum()

monthly_duration     288
monthly_messages     740
monthly_mb           269
monthly_gb           269
total_calls          288
state               2046
plan                2046
dtype: int64

In [84]:
monthly_stats.head()

monthly_duration  monthly_messages  monthly_mb  monthly_gb  \
user_id month                                                               
1000    12               116.83              11.0     1901.47     1.90147   
1001    8                171.14              30.0     6919.15     6.91915   
        9                297.69              44.0    13314.82    13.31482   
        10               374.11              53.0    22330.49    22.33049   
        11               404.59              36.0    18504.30    18.50430   

               total_calls state  plan  
user_id month                           
1000    12            16.0    GA   1.0  
1001    8             27.0    WA   0.0  
        9             49.0   NaN   NaN  
        10            65.0   NaN   NaN  
        11            64.0   NaN   NaN

In [85]:
monthly_stats['state'] = monthly_stats['state'].ffill()
monthly_stats['plan'] = monthly_stats['plan'].ffill()

In [86]:
#other missing values are due to sum being 0 
monthly_stats = monthly_stats.fillna(0)

In [87]:
monthly_stats.isna().sum()

monthly_duration    0
monthly_messages    0
monthly_mb          0
monthly_gb          0
total_calls         0
state               0
plan                0
dtype: int64

In [88]:
plans_data

,messages_included,mb_per_month_included,minutes_included,usd_monthly_pay,usd_per_gb,usd_per_message,usd_per_minute,plan_name,gb_per_month_included
0,50,15360,500,20,10,0.03,0.03,surf,15.36
1,1000,30720,3000,70,7,0.01,0.01,ultimate,30.72


### `Stats` by plan

In [89]:
surf_monthly_cost = 20
ulti_monthly_cost = 70
surf_mins_lim = 500
ulti_mins_lim = 3000
surf_mess_lim = 50
ulti_mess_lim = 1000
surf_gb_lim = 15.36
ulti_gb_lim = 30.72

In [90]:
stats_surf = monthly_stats.query('plan == 0').copy()
stats_ulti = monthly_stats.query('plan == 1').copy()


### mins over

In [91]:
stats_surf.loc[:, 'min_over'] = stats_surf['monthly_duration'] - surf_mins_lim
stats_ulti.loc[:, 'min_over'] = stats_ulti['monthly_duration'] - ulti_mins_lim

stats_surf.loc[stats_surf['min_over'] < 0, 'min_over'] = 0
stats_ulti.loc[stats_ulti['min_over'] < 0, 'min_over'] = 0

### messages over 

In [92]:
stats_surf.loc[:, 'mess_over'] = stats_surf['monthly_messages'] - surf_mess_lim
stats_ulti.loc[:, 'mess_over'] = stats_ulti['monthly_messages'] - ulti_mess_lim

stats_surf.loc[stats_surf['mess_over'] < 0, 'mess_over'] = 0
stats_ulti.loc[stats_ulti['mess_over'] < 0, 'mess_over'] = 0

### GB over

In [93]:
stats_surf.loc[:, 'gb_over'] = stats_surf['monthly_gb'] - surf_gb_lim
stats_ulti.loc[:, 'gb_over'] = stats_ulti['monthly_gb'] - ulti_gb_lim

stats_surf.loc[stats_surf['gb_over'] < 0, 'gb_over'] = 0
stats_ulti.loc[stats_ulti['gb_over'] < 0, 'gb_over'] = 0

### base cost for plan

In [95]:
stats_surf

monthly_duration  monthly_messages  monthly_mb  monthly_gb  \
user_id month                                                               
1001    8                171.14              30.0     6919.15     6.91915   
        9                297.69              44.0    13314.82    13.31482   
        10               374.11              53.0    22330.49    22.33049   
        11               404.59              36.0    18504.30    18.50430   
        12               392.93              44.0    19369.18    19.36918   
...                         ...               ...         ...         ...   
1489    5                  0.00               0.0        0.00     0.00000   
1492    3                  0.00               0.0        0.00     0.00000   
1494    4                  0.00               0.0        0.00     0.00000   
1496    2                  0.00               0.0        0.00     0.00000   
1499    5                  0.00               0.0        0.00     0.00000   

               total_calls     state  plan  min_over  mess_over  gb_over  
user_id month                                                             
1001    8             27.0        WA   0.0       0.0        0.0  0.00000  
        9             49.0        WA   0.0       0.0        0.0  0.00000  
        10            65.0        WA   0.0       0.0        3.0  6.97049  
        11            64.0        WA   0.0       0.0        0.0  3.14430  
        12            56.0        WA   0.0       0.0        0.0  4.00918  
...                    ...       ...   ...       ...        ...      ...  
1489    5              0.0        MI   0.0       0.0        0.0  0.00000  
1492    3              0.0     OR-WA   0.0       0.0        0.0  0.00000  
1494    4              0.0  NY-NJ-PA   0.0       0.0        0.0  0.00000  
1496    2              0.0        LA   0.0       0.0        0.0  0.00000  
1499    5              0.0        FL   0.0       0.0        0.0  0.00000  

[1651 rows x 10 columns]

In [99]:
stats_surf['base_cost']=20
stats_ulti['base_cost']=70

fixing the code starting here 

In [ ]:
stats['revenue'] = stats['min_over'] + stats['mess_over'] + stats['gb_over'] + stats['base']

In [ ]:
surf_data = stats.loc[stats['plan']== 0]
ult_data = stats.loc[stats['plan']== 1]

In [ ]:
stats.head()

In [ ]:
# Calculate the monthly revenue for each user
#total amount of time - revenue per month.
user_month_rev = stats.groupby(['user_id','month'])['revenue'].agg(total_rev = ('sum'))
                                                                    
user_month_rev.head()

In [ ]:
#total amount of time - calls per month.
user_month_min = stats.groupby(['user_id','month'])['duration'].agg(Sum = ('sum'))
user_month_min.head()

## Study user behaviour

### Calls

In [ ]:
stats.plan.astype('int')

In [ ]:
surf_monthly= stats.query("plan == 0")
ult_monthly= stats.query("plan == 1")

In [ ]:
ult_monthly.revenue

In [ ]:
# Compare the number of minutes users of each plan require each month. Plot a histogram.
surf_month_avg_min = surf_monthly.groupby(['month'])['duration'].agg(Surf_avg='mean')
ult_month_avg_min = ult_monthly.groupby(['month'])['duration'].agg(Ult_avg='mean')


In [ ]:
plan_avg_month_min = surf_month_avg_min.merge(ult_month_avg_min, on="month")
plan_avg_month_min.plot(kind='bar', title = 'Average Minute per Plan', width=.7,xlabel='Month', ylabel='Avg Minutes',grid=True)
plt.legend(loc=3)
plt.show()

In [ ]:
# Calculate the mean and the variance of the monthly call duration
plan_avg_month_min.mean()


In [ ]:
plan_avg_month_min.var()

In [ ]:
# Plot a boxplot to visualize the distribution of the monthly call duration
plan_avg_month_min.boxplot(showmeans=True, figsize=(3,4), widths=0.55, color='black')
plt.title('avg. min. used by plan')
plt.ylabel('minutes')
plt.show()


### calls - conclsion

surf plan users have a similar average minutes per month. however most are below the package limit.

how ever when looking at the distridutions of the data per plan the ultimte plans averages are higher and more tightly focused around 400 minutes. and none exceed the package limit. 

most users do not exceed there package limit for minutes. 

### Messages

In [ ]:
# Compare the number of messages users of each plan tend to send each month
surf_data.messages.mean()

In [ ]:
ult_data.messages.mean()

In [ ]:
surf_month_avg_mess =surf_monthly.groupby('month')['messages'].agg(Surf_avg='mean')
ult_month_avg_mess = ult_monthly.groupby('month')['messages'].agg(Ult_avg='mean')

In [ ]:
plan_avg_month_mess = surf_month_avg_mess.merge(ult_month_avg_mess, on="month")
plan_avg_month_mess.plot(kind='bar', title = 'Average messages per Month', width=.7,xlabel='Month', ylabel='Avg messages',grid=True)
plt.legend(loc=4)
plt.show()

In [ ]:
plan_avg_month_mess.boxplot(showmeans=True, widths=0.55, figsize=(3,4), color='black')
plt.title('messages sent by plan')
plt.ylabel('messages')
plt.show()

### messages - conclsion 

surf users send less messages per month than the ultimate users.

all users seem to fall below the limit of their respective package.

### Internet

In [ ]:
# Compare the amount of internet traffic consumed by users per plan
surf_month_avg_gb = surf_monthly.groupby(['month'])['gb'].agg(Surf_avg='mean')
ult_month_avg_gb = ult_monthly.groupby(['month'])['gb'].agg(Ult_avg='mean')

In [ ]:
plan_avg_month_gb = surf_month_avg_gb.merge(ult_month_avg_gb, on="month")
plan_avg_month_gb.round(1)

In [ ]:
plan_avg_month_gb.plot(kind='bar', title = 'average gb used per Plan', width=.8,xlabel='Month', ylabel='GB',grid=True)
plt.legend(loc=2)
plt.show()

In [ ]:
plan_avg_month_gb = surf_month_avg_gb.merge(ult_month_avg_gb, on="month")

In [ ]:
plan_avg_month_gb.boxplot(showmeans=True, widths=0.55, figsize=(3,4), color='black')
plt.title('gb used by plan')
plt.ylabel('GB')
plt.show()

In [ ]:
plan_avg_month_gb.mean()

### gb - conclsion 

surf users tend to use less data than the users of the ultimante plan 
however the surf users tend to exceed their plan more often. 

all users rarly use over 20 gb of data in a month. 

### Revenue

In [ ]:
surf_month_avg_rev = surf_monthly.groupby(['month'])['revenue'].agg(Surf_avg='mean')
ult_month_avg_rev = ult_monthly.groupby(['month'])['revenue'].agg(Ult_avg='mean')

In [ ]:
plan_avg_month_rev = surf_month_avg_rev.merge(ult_month_avg_rev, on="month")
plan_avg_month_rev.plot(kind='bar', title = 'Total Revenue per Plan', width=.8, xlabel='Month', ylabel='Revenue',grid=True)
plt.legend(loc=2)
plt.show()

In [ ]:
plan_avg_month_rev.boxplot(showmeans=True, widths=0.55, figsize=(3,4), color='black')
plt.title('Average Revenue used by plan')
plt.ylabel('Revenue($)')
plt.show()

In [ ]:
surf_total_rev = surf_data.revenue.sum()
surf_total_rev

In [ ]:
ult_total_rev = ult_data.revenue.sum()
ult_total_rev

In [ ]:
total_rev_surf = surf_data.revenue.count()
total_rev_surf

In [ ]:
total_rev_ult = ult_data.revenue.count()
total_rev_ult

In [ ]:
total_rev_ult/total_rev_surf * 100

[Formulate conclusions about how the revenue differs between the plans.]

In [ ]:
(ult_total_rev / surf_total_rev) * 100

### revenue - conclsion 

the Ultimante plan seem to make a steady amount around 70 dollars not alot over base. 

surf users pay around 50 dollars, which is 30$ more than than the base. meaning over halfof the revenue is made from overages. 

the surf plan in total generates more revenue overall due to having more users, however the average per user is much lower. 

the ultimante plan brings in more revenue per user while have less total revenue. 


# Test statistical hypotheses


### Hypothesis 

 **H0** 
    - There is *no diffrence* between the averges of both plans. 
    
**H1**
    - The mean values of the populations of each plan are *significantly diffrent*.

In [ ]:
sample_stats_surf = stats.query("plan == 0")

sample_stats_ulti = stats.query("plan == 1")

In [ ]:
rev_mean_surf = sample_stats_surf.revenue.mean()
rev_var_surf = sample_stats_surf.revenue.var()

In [ ]:
rev_mean_ulti = sample_stats_ulti.revenue.mean()
rev_var_ulti= sample_stats_ulti.revenue.var()

In [ ]:
print('surf varience:',rev_var_surf , "Ultimate varience:" , rev_var_ulti)

In [ ]:
plan_rev_result = py.stats.ttest_ind(sample_stats_surf['revenue'], sample_stats_ulti['revenue'], equal_var=False)
plan_rev_result.pvalue

In [ ]:
alpha=0.02

In [ ]:
# Test the hypotheses
print('p-value:', plan_rev_result.pvalue / 2)

if (plan_rev_result.pvalue / 2) < alpha:
    print("the null hypothesis should be rejected")
else:
    print("the null hypthesis can not be rejected")

In [ ]:
sample_stats_NY = stats.query("state in 'NY'")
sample_stats_other = stats.query("state not in 'NY'")

In [ ]:
rev_mean_NY = sample_stats_NY.revenue.mean()
rev_var_NY= sample_stats_NY.revenue.var()

In [ ]:
rev_mean_other = sample_stats_other.revenue.mean()
rev_var_other= sample_stats_other.revenue.var()

In [ ]:
print('NY area varience:',rev_var_NY , "other area varience:" , rev_var_other)

In [ ]:
rev_var_other/rev_var_NY 

In [ ]:
NY_rev_results = py.stats.ttest_ind(sample_stats_NY['revenue'], sample_stats_other['revenue'], equal_var=True)
NY_rev_results.pvalue

In [ ]:
# Test the hypotheses
print('p-value:', NY_rev_results.pvalue / 2)

if (NY_rev_results.pvalue / 2) < alpha:
    print("the null hypothesis should be rejected")
else:
    print("the null hypthesis can not be rejected")

Users from the NY-NJ area do not differ from the other states enough to reject the null hypothesis that the datasets are not equal.  

# General conclusion

**Metrics Conclusion**

- Minutes
    - users from both plans tend to stay below their call limits. 
- Messages 
    - again, users dont often go over the plans limit. 
- GB
    - Data is where users tend to exceed the limits of the given plan. 
        - this is extremely common amoung surf plan users. 
        - data overage fees are the main source of revenue for the whole plan. 
    - users rarely exceed 20 gb which is below the ultimante plans limit.
    
**Plan Conclusions** 

- *Surf* currently brings in more revenue 
    - at roughly ~100,000 or 51% of the overall revenue.
    - a result of having 40% more users then Ultimate. 

- *Ultimante* is the more profitable plan. 
    - making up 49% of the total revenue. 
    - while having only a quarter of the total users.
    
**Final Conclusion** 

- The plan which generates **the most revenue is the Ultimante plan**. 
    - **the biggest factor in each plan is the base costs.** dispite the fact the many surf users exceed their data limit they still pay less than the base cost of an ultimante user.

